# Residual Network Ablation Study

## 1. Import Libaray

In [4]:
import tensorflow as tf
import numpy as np

print(tf.__version__)
print(np.__version__)

2.16.2
1.26.4


## 2. ConvNet Block

In [164]:
def conv_block(num_layer, filters, kernel_sizes, padding="same", stride=1):


    layers = []

    filters = [filters] * num_layer if type(filters) in [int, tuple] else filters
    kernel_sizes = [kernel_sizes] * num_layer if type(kernel_sizes) in [int, tuple] else kernel_sizes

    assert num_layer == len(filters) == len(kernel_sizes)
    
    print("conv block filters", filters[0])
    for i in range(num_layer):
        conv = tf.keras.layers.Conv2D(filters=filters[i],
                        kernel_size=kernel_sizes[i],
                        padding=padding,
                        strides=stride,
                        name=f"conv {i+1}")
        layers.append(conv)
        stride = 1
    return tf.keras.Sequential(layers)

In [165]:
block = conv_block(num_layer=5, filters=4, kernel_sizes=(3, 3))

inputs = tf.keras.layers.Input(shape=(224, 224, 3))
outputs = block(inputs)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.summary()

conv block filters 4


Model: "functional_92"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_128 (InputLayer)    │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_106 (Sequential)     │ (None, 224, 224, 4)    │           704 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 704 (2.75 KB)

 Trainable params: 704 (2.75 KB)

 Non-trainable params: 0 (0.00 B)

## 3. Module Block

### 1) pooling module

In [166]:
def conv_pool_block(num_layers, filters, kernel_size, pool_sizes, padding="same", stride=(1, 1)):
    
    layers = []
    
    assert len(num_layers) == len(filters) == len(pool_sizes)
    
    for i in range(len(num_layers)):
        block = conv_block(num_layer=num_layers[i],
                                filters=filters[i],
                                kernel_sizes=kernel_size,
                                padding=padding,
                                stride=stride)
        layers.append(block)
        pool = tf.keras.layers.MaxPooling2D(pool_size=pool_sizes[i])
        layers.append(pool)
    
    sequence = tf.keras.Sequential(layers)
    return sequence
    

In [167]:
num_layers = [2, 4, 4]
filters = [64, 128, 256]
pool_sizes = [2, 2, 2]

pool_module = conv_pool_block(num_layers=num_layers,
                            filters=filters,
                            pool_sizes=pool_sizes,
                            kernel_size=(3, 3))

inputs = tf.keras.layers.Input(shape=(224, 224, 3))
outputs = pool_module(inputs)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.summary()

conv block filters 64
conv block filters 128
conv block filters 256


Model: "functional_97"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_130 (InputLayer)    │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_110 (Sequential)     │ (None, 28, 28, 256)    │     2,620,736 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,620,736 (10.00 MB)

 Trainable params: 2,620,736 (10.00 MB)

 Non-trainable params: 0 (0.00 B)

### 2) stride module

In [168]:
def conv_residual_block(inputs, 
                        num_block, 
                        num_layers, 
                        filters, 
                        kernel_sizes, 
                        padding="same", 
                        stride=1,
                        plain=False):
    x = inputs
    
    filters = [filters] * num_block if type(filters) in [int, tuple] else filters
    kernel_sizes = [kernel_sizes] * num_block if type(kernel_sizes) in [int, tuple] else kernel_sizes
    
    for i in range(num_block):
        print("num_layers[i]", num_layers[i])
        print("filters[i]", filters[i])
        print("kernel_sizes[i]", kernel_sizes[i])
        print("padding", padding)
        print("stride", stride)
        conv = conv_block(num_layer=num_layers[i], 
                    filters=filters[i], 
                    kernel_sizes=kernel_sizes[i],
                    padding=padding,
                    stride=stride)
        x = conv(x)
        stride = 1
        print("x", x.shape)
        print("inputs", inputs.shape)
        print("\n")
        x if plain else x + inputs

    return x

In [169]:
def residual_layer(inputs,
                   num_blocks,
                   num_layers,
                   filters,
                   kernel_sizes,
                   padding="same",
                   strides=1,
                   plain=False):
    x = inputs
    len_block = len(num_blocks)
    kernel_sizes = [kernel_sizes] * len_block if type(kernel_sizes) in [int, tuple] else kernel_sizes

    for i in range(len_block):
        x = conv_residual_block(inputs=x,
                                             num_block=num_blocks[i],
                                             num_layers=num_layers,
                                             filters=filters[i],
                                             kernel_sizes=kernel_sizes[i],
                                             padding=padding,
                                             stride=strides[i],
                                             plain=plain)
    return x

In [1]:
def build_resnet(input_shape, type, plain=False):
    
    inputs = tf.keras.layers.Input(shape=input_shape)
    
    conv = tf.keras.layers.Conv2D(filters=64, kernel_size=7, padding="same", strides=2)
    x = conv(inputs)
    print("start x", x.shape)
    pool = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)
    x = pool(x)
    print("start pool x", x.shape)
    
    if type == "is_34":
        num_blocks = [3, 4, 6, 3]
        num_layers = [2, 2, 2, 2]
        filters = [64, 128, 256, 512]
        kernel_sizes = 3
        strides = [1, 2, 2, 2]
    
    elif type == "is_50":
        num_blocks = [3, 4, 6, 3]
        num_layers = [3, 3, 3, 3]
        filters = [[64, 64, 256], 
                   [128, 128, 512], 
                   [256, 256, 1024],
                   [512, 512, 2048]]
        kernel_sizes = [[1, 3, 1]]
        strides = [1, 2, 2, 2]
        
    layers = residual_layer(inputs=x,
                                 num_blocks=num_blocks,
                                 num_layers=num_layers,
                                 filters=filters,
                                 kernel_sizes=kernel_sizes,
                                 padding="same",
                                 strides=strides,
                                 plain=plain)
    x = layers(x)
    
    global_pool = tf.keras.layers.GlobalAveragePooling2D()
    x = global_pool(x)
    
    classifier = tf.keras.layers.Dense(1000, activation="softmax")
    outputs = classifier(x)
    return outputs